In [1]:
"""для теста датасет будет использоваться обычный титаник
D:\Py_things\PyProjects\titanik_train_Project\titanic.csv
"""

'для теста датасет будет использоваться обычный титаник\nD:\\Py_things\\PyProjects\titanik_train_Project\titanic.csv\n'

In [2]:
%env CLEARML_WEB_HOST=http://clearml.spb.rdi-kvant.ru:8080
%env CLEARML_API_HOST=http://clearml.spb.rdi-kvant.ru:8008
%env CLEARML_FILES_HOST=http://clearml.spb.rdi-kvant.ru:8081
%env CLEARML_API_ACCESS_KEY=W3W1A5F3FC2ZLJC95HAM
%env CLEARML_API_SECRET_KEY=0ratpE3DCGUDMwFiuzE3pOcuwKIKiihnfG2vgchVFteE8bX7Al

env: CLEARML_WEB_HOST=http://clearml.spb.rdi-kvant.ru:8080
env: CLEARML_API_HOST=http://clearml.spb.rdi-kvant.ru:8008
env: CLEARML_FILES_HOST=http://clearml.spb.rdi-kvant.ru:8081
env: CLEARML_API_ACCESS_KEY=W3W1A5F3FC2ZLJC95HAM
env: CLEARML_API_SECRET_KEY=0ratpE3DCGUDMwFiuzE3pOcuwKIKiihnfG2vgchVFteE8bX7Al


In [3]:
from clearml import Task, Logger
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, ParameterSampler
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score

In [4]:
task = Task.init(project_name='OTUS_test',
           task_name='CatBoost_notebook_v1',
           tags=['Model:CatBoost', 'type:notebook'])

ClearML Task: created new task id=c793a77bfacf4f10b93f7ffc1e88d7a5
ClearML results page: http://clearml.spb.rdi-kvant.ru:8080/projects/2c29146474784712a23a87a4be54b4a4/experiments/c793a77bfacf4f10b93f7ffc1e88d7a5/output/log


In [5]:
df_raw = pd.read_csv('D:\\Py_things\\PyProjects\\titanik_train_Project\\titanic.csv')

task.upload_artifact(name='data.raw', artifact_object=df_raw)

df_raw.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
task.upload_artifact(name='eda describe', artifact_object=df_raw.describe())
df_raw.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
task.upload_artifact(name='target_info', artifact_object=df_raw['Survived'].value_counts(normalize=True).reset_index())

True

In [8]:
df_preproc = df_raw.drop(columns=['PassengerId', 'Name', 'Ticket'])
for col in ['Sex', 'Cabin', 'Embarked']:
    df_preproc[col] = df_preproc[col].astype(str)
    
task.upload_artifact(name='data.preproc', artifact_object=df_preproc)

True

In [9]:
df_raw.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [10]:
train, test = train_test_split(df_preproc, test_size=0.33, random_state=42)

task.upload_artifact(name='data.train', artifact_object=train)
task.upload_artifact(name='data.test', artifact_object=test)

True

In [11]:
X_train = train.drop(columns=['Survived']) 
X_train = X_train[['Sex', 'Cabin', 'Embarked']]
y_train = train['Survived']

X_test = test.drop(columns=['Survived']) 
X_test = X_test[['Sex', 'Cabin', 'Embarked']]
y_test = test['Survived']

In [12]:
param_grid = {
    'depth' : [1, 2, 16],
    'learning_rate': [0.1, 0.0001],
    'iterations': [10, 20, 30]
}

log = Logger.current_logger()

best_score = 0 
best_model = None
i = 0

In [13]:
for param in ParameterSampler(param_grid, n_iter=4, random_state=42):
    print(param)
    parametrs_dict = Task.current_task().connect(param)
    print(i+1)
    
    model = CatBoostClassifier(**param, silent=True)
    model.fit(X_test, y_test, cat_features=['Sex', 'Cabin', 'Embarked'])
    
    #Оценка модели и обучение 
    test_scores = model.eval_metrics(
    data=Pool(X_test, y_test, cat_features=['Sex', 'Cabin', 'Embarked']),
    metrics=['Logloss', 'AUC'])
    test_logloss = round(test_scores['Logloss'][-1], 4)
    test_roc_auc = round(test_scores['AUC'][-1]*100, 4)
    
    train_scores = model.eval_metrics(
    data=Pool(X_train, y_train, cat_features=['Sex', 'Cabin', 'Embarked']),
    metrics=['Logloss', 'AUC'])
    train_logloss = round(train_scores['Logloss'][-1], 4)
    train_roc_auc = round(train_scores['AUC'][-1]*100, 4)
    
    
    if test_roc_auc > best_score:
        best_score = test_roc_auc
        best_model = model
        
        log.report_scalar('Logloss', 'Test', iteration=i, value=test_logloss)
        log.report_scalar('Logloss', 'Train', iteration=i, value=train_logloss)
        
        log.report_scalar('ROC AUC', 'Test', iteration=i, value=test_roc_auc)
        log.report_scalar('ROC AUC', 'Train', iteration=i, value=train_roc_auc)
        
        i+=1

{'learning_rate': 0.1, 'iterations': 10, 'depth': 1}
1
{'learning_rate': 0.0001, 'iterations': 10, 'depth': 1}
ClearML Monitor: GPU monitoring failed getting GPU reading, switching off GPU monitoring
2
{'learning_rate': 0.1, 'iterations': 20, 'depth': 2}
2
{'learning_rate': 0.0001, 'iterations': 30, 'depth': 1}
3
2024-02-29 17:19:51,854 - clearml.Task - INFO - Storing jupyter notebook directly as code


In [14]:
log.report_single_value(name='Best ROC AUC', value=test_roc_auc)
log.report_single_value(name='Best Logloss', value=test_logloss)
log.report_single_value(name='Train Rows', value=X_train.shape[0])
log.report_single_value(name='Test Rows', value=X_test.shape[0])
log.report_single_value(name='Columns', value=X_train.shape[1])

In [16]:
best_model_name = 'webinar_madel_v2.cbn'
best_model.save_model(best_model_name)
task.upload_artifact(name=best_model_name, artifact_object=best_model)

True

In [17]:
task.close()